# TPOT

In [1]:
%run notebooks/04-feature-engineering.ipynb

print("START: 06-TPOT.ipynb")

01-data-labelling.ipynb
02-data-cleaning.ipynb
  Dropping AgeinYears, CTDone, CTForm1, CTSed, CTSedAge, CTSedAgitate, CTSedOth, CTSedRqst, Certification, DeathTBI, EDCT, EDDisposition, EmplType, Finding1, Finding10, Finding11, Finding12, Finding13, Finding14, Finding2, Finding20, Finding21, Finding22, Finding23, Finding3, Finding4, Finding5, Finding6, Finding7, Finding8, Finding9, GCSGroup, High_impact_InjSev, HospHead, HospHeadPosCT, IndAMS, IndAge, IndAmnesia, IndClinSFx, IndHA, IndHema, IndLOC, IndMech, IndNeuroD, IndOth, IndRqstMD, IndRqstParent, IndRqstTrauma, IndSeiz, IndVomit, IndXraySFx, Intub24Head, Neurosurgery, Observed, PosCT
  PosIntFinal - Dropping 20 rows where PosIntFinal is NaN
  Gender - Dropping 3 rows where Gender is NaN
  Filling missing GCSEye, GCSVerbal, GCSMotor when GCSTotal is 15
  GCSTotal - Dropping as it is now redundant
  AgeInMonths - Renaming to Age
  Renaming InjuryMech to Injury_Mechanism
  Renaming ActNorm to Acting_Normal
03-data-imputation.ipynb
  T

## Arrange Data
Need to convert the dataframes to numpy arrays so that TPOT can work with them.

In [2]:
features = pecarn_imputed.copy()

label = features['PosIntFinal']
label = features['PosIntFinal'].cat.codes
label = label.to_numpy(dtype=np.int32)

attributes = features.drop(columns='PosIntFinal')
for col in attributes:
    if attributes[col].dtype == 'category':
        attributes[col] = attributes[col].cat.codes
attributes = attributes.to_numpy(dtype=np.int32)

In [3]:
from sklearn.model_selection import train_test_split

#features = pecarn_cleaned.copy()

#y = features['PosIntFinal']
#X = features.drop(columns='PosIntFinal')
y = label
X = attributes

# create training and test splits - this will act as a hold out set
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, test_size=0.25, stratify=y)


# Configure TPOT
Initial attempts to use the TPOT Light template were taking a long time too complete.

We create a tpot_config that is based on the TPOT Light template, and removing in particular preprocessors that we aren't interested in.

See https://github.com/EpistasisLab/tpot/blob/master/tpot/config/classifier_light.py

In [4]:
# https://github.com/EpistasisLab/tpot/blob/master/tpot/config/classifier_light.py
# https://github.com/EpistasisLab/tpot/blob/master/tpot/config/classifier.py

# specify the pipelines that TPOT will optmize - selected from the two default classifiers
tpot_config = {

    # # Classifiers
    # 'sklearn.naive_bayes.GaussianNB': {
    # },

    # 'sklearn.naive_bayes.BernoulliNB': {
    #     'alpha': [1e-3, 1e-2, 1e-1, 1., 10., 100.],
    #     'fit_prior': [True, False]
    # },

    # 'sklearn.naive_bayes.MultinomialNB': {
    #     'alpha': [1e-3, 1e-2, 1e-1, 1., 10., 100.],
    #     'fit_prior': [True, False]
    # },

    'sklearn.tree.DecisionTreeClassifier': {
        'criterion': ["gini", "entropy"],
        'max_depth': range(1, 11),
        'min_samples_split': range(2, 21),
        'min_samples_leaf': range(1, 21)
    },

    # 'sklearn.ensemble.ExtraTreesClassifier': {
    #     'n_estimators': [100],
    #     'criterion': ["gini", "entropy"],
    #     'max_features': np.arange(0.05, 1.01, 0.05),
    #     'min_samples_split': range(2, 21),
    #     'min_samples_leaf': range(1, 21),
    #     'bootstrap': [True, False]
    # },

    # 'xgboost.XGBClassifier': {
    #     'n_estimators': [100],
    #     'max_depth': range(1, 11),
    #     'learning_rate': [1e-3, 1e-2, 1e-1, 0.5, 1.],
    #     'subsample': np.arange(0.05, 1.01, 0.05),
    #     'min_child_weight': range(1, 21),
    #     'nthread': [1]
    # },

    # 'sklearn.neighbors.KNeighborsClassifier': {
    #     'n_neighbors': range(1, 101),
    #     'weights': ["uniform", "distance"],
    #     'p': [1, 2]
    # },


    # 'sklearn.linear_model.LogisticRegression': {
    #     'penalty': ["l1", "l2"],
    #     'C': [1e-4, 1e-3, 1e-2, 1e-1, 0.5, 1., 5., 10., 15., 20., 25.],
    #     'dual': [True, False]
    # },

    # Preprocesssors
    # 'sklearn.cluster.FeatureAgglomeration': {
    #     'linkage': ['ward', 'complete', 'average'],
    #     'affinity': ['euclidean', 'l1', 'l2', 'manhattan', 'cosine']
    # },

    # 'sklearn.decomposition.PCA': {
    #     'svd_solver': ['randomized'],
    #     'iterated_power': range(1, 11)
    # },

    # 'sklearn.kernel_approximation.RBFSampler': {
    #     'gamma': np.arange(0.0, 1.01, 0.05)
    # },

    # 'sklearn.preprocessing.RobustScaler': {
    # },

    # 'sklearn.preprocessing.StandardScaler': {
    # },

    'sklearn.preprocessing.OneHotEncoder': {
        'drop': 'first'
    }

    # 'tpot.builtins.OneHotEncoder': {
    #     'minimum_fraction': [0.05, 0.1, 0.15, 0.2, 0.25],
    #     'sparse': [False],
    #     'threshold': [10]
    # },

    # Selectors
    # 'sklearn.feature_selection.SelectFwe': {
    #     'alpha': np.arange(0, 0.05, 0.001),
    #     'score_func': {
    #         'sklearn.feature_selection.f_classif': None
    #     }
    # },

    # 'sklearn.feature_selection.VarianceThreshold': {
    #     'threshold': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.2]
    # }

    # 'sklearn.feature_selection.SelectFromModel': {
    #     'threshold': np.arange(0, 1.01, 0.05),
    #     'estimator': {
    #         'sklearn.ensemble.ExtraTreesClassifier': {
    #             'n_estimators': [100],
    #             'criterion': ['gini', 'entropy'],
    #             'max_features': np.arange(0.05, 1.01, 0.05)
    #         }
    #     }
    # }

}

from tpot import TPOTClassifier

# create TPOT optimizer
clf_opt = TPOTClassifier(
                        #config_dict='TPOT light',
                        scoring='f1',
                        
                        #generations=10,
                        #population_size=50, 
                        #memory='auto',
                        n_jobs=-1,
                        #early_stop=3,
                        verbosity=2)

## Call TPOT

In [5]:
clf_opt.fit(X_train, y_train)

Optimization Progress:  91%|█████████ | 500/550 [02:02<01:02,  1.24s/pipeline]Generation 9 - Current best internal CV score: 0.4998521465697728

The optimized pipeline was not improved after evaluating 3 more generations. Will end the optimization process.

TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: DecisionTreeClassifier(CombineDFs(DecisionTreeClassifier(input_matrix, criterion=gini, max_depth=4, min_samples_leaf=20, min_samples_split=16), input_matrix), criterion=gini, max_depth=1, min_samples_leaf=1, min_samples_split=17)


TPOTClassifier(config_dict={'sklearn.preprocessing.OneHotEncoder': {'drop': 'first'},
                            'sklearn.tree.DecisionTreeClassifier': {'criterion': ['gini',
                                                                                  'entropy'],
                                                                    'max_depth': range(1, 11),
                                                                    'min_samples_leaf': range(1, 21),
                                                                    'min_samples_split': range(2, 21)}},
               crossover_rate=0.1, cv=5, disable_update_check=False,
               early_stop=3, generations=10, max_eval_time_mins=5,
               max_time_mins=None, memory=None, mutation_rate=0.9, n_jobs=-1,
               offspring_size=None, periodic_checkpoint_folder=None,
               population_size=50, random_state=None, scoring='f1',
               subsample=1.0, template=None, use_dask=False, verbosity=2,
   

## Score 

In [6]:
print(clf_opt.score(X_test, y_test))

0.4359861591695502


In [7]:
clf_opt.fitted_pipeline_

Pipeline(memory=None,
         steps=[('featureunion',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('stackingestimator',
                                                 StackingEstimator(estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                                                                    class_weight=None,
                                                                                                    criterion='gini',
                                                                                                    max_depth=4,
                                                                                                    max_features=None,
                                                                                                    max_leaf_nodes=None,
                                                                                                    min_impurity_decrease=0.0,


## Export Pipeline

In [8]:
clf_opt.export('pipelines/tpot_tbi_pipeline.py')